In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from tqdm import tqdm  # Importar tqdm para la barra de progreso

# Cargar el modelo y el tokenizador
model_name = "FreedomIntelligence/HuatuoGPT-o1-7B"
device = "cuda" if torch.cuda.is_available() else "cpu"

# Inicializar el modelo y el tokenizador
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", torch_dtype=torch.float16).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Función para realizar razonamiento con barra de progreso
def razonamiento_medico_con_progreso(texto1, texto2):
    prompt = f"""
    Dado los siguientes resultados médicos de un mismo paciente, determina cuál es la patología más probable considerando las probabilidades asociadas. Ten en cuenta que el primer texto viene de un modelo de texto y el segundo viene de un modelo de imágen.

    Resultados modelo de texto 1:
    {texto1}

    Resultados modelo de imágen 2:
    {texto2}

    Proporciona un razonamiento paso a paso para llegar a la conclusión. Y después dame un pequeño documento con las conclusiones médicas de la patología y siguientes pasos que hacer con el paciente en función de su principal patología.
    """

    # Tokenizar el prompt
    inputs = tokenizer(prompt, return_tensors="pt", max_length=1024, truncation=True).to(device)

    # Configurar la barra de progreso
    with tqdm(total=100, desc="Procesando razonamiento médico") as pbar:
        # Generar respuesta del modelo
        outputs = model.generate(
            inputs.input_ids,
            max_new_tokens=3000,
            temperature=0.7,
            top_p=0.9,
            do_sample=True,
            num_return_sequences=1
        )
        pbar.update(100)  # Actualizar la barra al 100% una vez completado

    # Decodificar la respuesta generada
    respuesta = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return respuesta




/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/686 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/27.8k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.88G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.93G [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/605 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/613 [00:00<?, ?B/s]

Truth:
* Enlarged Cardiomediastinum: Negativo
* Cardiomegaly: Negativo
* Lung Opacity: Negativo
* Lung Lesion: Negativo
* Edema: Incierto
* Consolidation: Incierto
* Pneumonia: Negativo
* Atelectasis: Negativo
* Pneumothorax: Negativo
* Pleural Effusion: Positivo
* Pleural Other: Negativo
* Fracture: Negativo

In [ ]:
# Ejemplo de entrada
texto1 = """Enlarged Cardiomediastinum: Negativo (Prob: 0.1412)
Cardiomegaly: Negativo (Prob: 0.0919)
Lung Opacity: Positivo (Prob: 0.5736)
Lung Lesion: Negativo (Prob: 0.0326)
Edema: Negativo (Prob: 0.2209)
Consolidation: Negativo (Prob: 0.0679)
Pneumonia: Negativo (Prob: 0.0197)
Atelectasis: Negativo (Prob: 0.1764)
Pneumothorax: Negativo (Prob: 0.0656)
Pleural Effusion: Incierto (Prob: 0.4574)
Pleural Other: Negativo (Prob: 0.0093)
Fracture: Negativo (Prob: 0.0322)"""

texto2 = """Enlarged Cardiomediastinum: Probability = 0.0168, Prediction = 0.0
Cardiomegaly: Probability = 0.0584, Prediction = 0.0
Lung Opacity: Probability = 0.4243, Prediction = 0.0
Lung Lesion: Probability = 0.0160, Prediction = 0.0
Edema: Probability = 0.2896, Prediction = 0.0
Consolidation: Probability = 0.0252, Prediction = 0.0
Pneumonia: Probability = 0.0074, Prediction = 0.0
Atelectasis: Probability = 0.2245, Prediction = 0.0
Pneumothorax: Probability = 0.0522, Prediction = 0.0
Pleural Effusion: Probability = 0.6251, Prediction = 1.0
Pleural Other: Probability = 0.0024, Prediction = 0.0
Fracture: Probability = 0.0075, Prediction = 0.0"""

# Ejecutar razonamiento médico con barra de progreso
respuesta = razonamiento_medico_con_progreso(texto1, texto2)

# Mostrar resultado
print("Respuesta del modelo:")
print(respuesta)

Procesando razonamiento médico: 100%|██████████| 100/100 [03:16<00:00,  1.97s/it]

Respuesta del modelo:

    Dado los siguientes resultados médicos de un mismo paciente, determina cuál es la patología más probable considerando las probabilidades asociadas. Ten en cuenta que el primer texto viene de un modelo de texto y el segundo viene de un modelo de imágen.

    Resultados modelo de texto 1:
    Enlarged Cardiomediastinum: Negativo (Prob: 0.1412)
Cardiomegaly: Negativo (Prob: 0.0919)
Lung Opacity: Positivo (Prob: 0.5736)
Lung Lesion: Negativo (Prob: 0.0326)
Edema: Negativo (Prob: 0.2209)
Consolidation: Negativo (Prob: 0.0679)
Pneumonia: Negativo (Prob: 0.0197)
Atelectasis: Negativo (Prob: 0.1764)
Pneumothorax: Negativo (Prob: 0.0656)
Pleural Effusion: Incierto (Prob: 0.4574)
Pleural Other: Negativo (Prob: 0.0093)
Fracture: Negativo (Prob: 0.0322)

    Resultados modelo de imágen 2:
    Enlarged Cardiomediastinum: Probability = 0.0168, Prediction = 0.0
Cardiomegaly: Probability = 0.0584, Prediction = 0.0
Lung Opacity: Probability = 0.4243, Prediction = 0.0
Lung Les